<a href="https://colab.research.google.com/github/Jirapat2035/bsc_dpdm24/blob/main/%E0%B8%99%E0%B8%B2%E0%B8%A2%E0%B8%88%E0%B8%B4%E0%B8%A3%E0%B8%A0%E0%B8%B1%E0%B8%97%E0%B8%A3_%E0%B9%81%E0%B8%AA%E0%B8%87%E0%B8%AA%E0%B8%B8%E0%B8%81%E0%B9%83%E0%B8%AA_653020203_5_midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

import os
os.chdir('/content/drive/MyDrive/bsc_dpdm24_data')

In [5]:
Data_House = pd.read_excel("/content/drive/MyDrive/bsc_dpdm24_data/Data House.xlsx")

In [6]:
Data_House

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# เลือกคอลัมน์ที่ต้องการ
columns = ['renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
           'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
           'direction', 'furniture', 'usable_area', 'number_of_parking',
           'sell_price', 'district', 'sub_district', 'zone','latitude', 'longitude']
Data_House = Data_House[columns]

In [24]:
# จัดการ Missing Values
Data_House.fillna({
    'usable_area': Data_House['usable_area'].mean(),
    'number_of_bedroom': Data_House['number_of_bedroom'].median(),
    'direction': Data_House['direction'].mode()[0]
}, inplace=True)

In [25]:
# กำจัด Outliers
Q1 = Data_House['sell_price'].quantile(0.25)
Q3 = Data_House['sell_price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data_house = Data_House[(Data_House['sell_price'] >= lower_bound) & (Data_House['sell_price'] <= upper_bound)]


In [26]:
# Normalization
scaler = MinMaxScaler()
Data_House[['usable_area', 'sell_price', 'number_of_parking']] = scaler.fit_transform(Data_House[['usable_area', 'sell_price', 'number_of_parking']])

In [27]:
Data_House.describe()

,land wide(m),land long(m),zip_code,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,construct_year,model,sell_price,number_of_story,number_of_bedroom,number_of_bathroom,number_of_parking,usable_area,central_cost,floor_number
count,56.000000,47.000000,1.181000e+03,1634.000000,1634.000000,1656.000000,1656.000000,1656.000000,0.0,0.0,1655.000000,1107.000000,1656.000000,1656.000000,1441.000000,1656.000000,1032.000000,297.000000
mean,15.225000,22.665957,6.813955e+06,16.816693,103.043810,0.350242,0.368357,58.740990,NaN,NaN,0.034226,1.513098,1.725242,1.738527,0.042489,0.007051,70.597868,0.030303
std,10.047202,19.252569,1.645379e+08,6.108615,5.905237,3.082830,2.424360,58.729835,NaN,NaN,0.044771,1.109942,2.367961,1.068580,0.068767,0.025878,374.496590,0.422058
min,4.000000,6.000000,4.000000e+00,13.752233,100.493675,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,8.725000,12.200000,4.000000e+04,16.397887,102.730874,0.000000,0.000000,26.000000,NaN,NaN,0.014111,1.000000,0.000000,1.000000,0.000000,0.004535,0.000000,0.000000
50%,13.000000,17.000000,4.000000e+04,16.434853,102.801100,0.000000,0.000000,58.000000,NaN,NaN,0.024714,2.000000,2.000000,1.000000,0.000000,0.007051,0.000000,0.000000
75%,16.625000,25.000000,4.014000e+04,16.488421,102.841940,0.000000,0.000000,74.575000,NaN,NaN,0.038811,2.000000,3.000000,2.000000,0.090909,0.007051,0.000000,0.000000
max,60.000000,110.000000,4.000040e+09,111.000000,222.000000,67.000000,95.000000,786.000000,NaN,NaN,1.000000,32.000000,40.000000,14.000000,1.000000,1.000000,7598.000000,7.000000


In [34]:
import pandas as pd
import json
import math

# ฟังก์ชันคำนวณระยะทาง (Haversine Formula)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# โหลดข้อมูลบ้าน
Data_House = pd.read_excel("/content/drive/MyDrive/bsc_dpdm24_data/Data House.xlsx")

# โหลดข้อมูล 7-11 และโรงเรียน
with open("/content/drive/MyDrive/bsc_dpdm24_data/7-eleven.json", "r") as f:
    seven_data = json.load(f)

with open("/content/drive/MyDrive/bsc_dpdm24_data/school.json", "r") as f:
    school_data = json.load(f)

# ตรวจสอบประเภทของข้อมูล
print(f"Type of seven_data: {type(seven_data)}")
print(f"Type of school_data: {type(school_data)}")

# ถ้าข้อมูลเป็นลิสต์, แสดงตัวอย่าง 5 รายการแรก
if isinstance(seven_data, list):
    print("7-Eleven Data Sample:")
    for i, shop in enumerate(seven_data[:5]):  # แสดง 5 รายการแรกจาก seven_data
        print(f"Sample {i+1}: {shop}")

if isinstance(school_data, list):
    print("\nSchool Data Sample:")
    for i, school in enumerate(school_data[:5]):  # แสดง 5 รายการแรกจาก school_data
        print(f"Sample {i+1}: {school}")
# สร้างคอลัมน์ใหม่ในข้อมูลบ้าน
Data_House['d1000-7/11'] = 0
Data_House['d1000-school'] = 0

# คำนวณจำนวน 7-11 ในรัศมี 1 กิโลเมตร
for idx, house in data_house.iterrows():
    house_lat = house['latitude']
    house_lon = house['longitude']

    # นับจำนวน 7-11 ในรัศมี 1 กิโลเมตร
count_7_eleven = sum(
        1 for shop in seven_data
        if isinstance(shop, dict) and 'latitude' in shop and 'longitude' in shop and haversine(house_lat, house_lon, shop['latitude'], shop['longitude']) <= 1
    )
Data_House.at[idx, 'd1000-7/11'] = count_7_eleven

    # นับจำนวนโรงเรียนในรัศมี 1 กิโลเมตร
count_school = sum(
        1 for school in school_data
        if isinstance(school, dict) and 'latitude' in school and 'longitude' in school and haversine(house_lat, house_lon, school['latitude'], school['longitude']) <= 1
    )
Data_House.at[idx, 'd1000-school'] = count_school

# แสดงผลข้อมูลหลังคำนวณ
Data_House[['d1000-7/11', 'd1000-school']]


Type of seven_data: <class 'dict'>
Type of school_data: <class 'dict'>


,d1000-7/11,d1000-school
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
1651,0,0
1652,0,0
1653,0,0
1654,0,0


In [38]:
import pandas as pd
import json
import math

# ฟังก์ชันคำนวณระยะทาง (Haversine Formula)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# โหลดข้อมูลบ้าน
Data_House = pd.read_excel("/content/drive/MyDrive/bsc_dpdm24_data/Data House.xlsx")

# โหลดข้อมูล 7-11 และโรงเรียน
with open("/content/drive/MyDrive/bsc_dpdm24_data/7-eleven.json", "r") as f:
    seven_data = json.load(f)

with open("/content/drive/MyDrive/bsc_dpdm24_data/school.json", "r") as f:
    school_data = json.load(f)

# ตรวจสอบประเภทของข้อมูล
print(f"Type of seven_data: {type(seven_data)}")
print(f"Type of school_data: {type(school_data)}")

# ถ้าข้อมูลเป็นลิสต์, แสดงตัวอย่าง 5 รายการแรก
if isinstance(seven_data, list):
    print("7-Eleven Data Sample:")
    for i, shop in enumerate(seven_data[:5]):  # แสดง 5 รายการแรกจาก seven_data
        print(f"Sample {i+1}: {shop}")

if isinstance(school_data, list):
    print("\nSchool Data Sample:")
    for i, school in enumerate(school_data[:5]):  # แสดง 5 รายการแรกจาก school_data
        print(f"Sample {i+1}: {school}")

# สร้างคอลัมน์ใหม่ในข้อมูลบ้าน
Data_House['d1000-7/11'] = 0
Data_House['d1000-school'] = 0

# คำนวณจำนวน 7-11 ในรัศมี 1 กิโลเมตร
for idx, house in Data_House.iterrows():
    house_lat = house['latitude']
    house_lon = house['longitude']

    # นับจำนวน 7-11 ในรัศมี 1 กิโลเมตร
    count_7_eleven = sum(
        1 for shop_id, shop in seven_data.items()  # ใช้ .items() สำหรับ dict
        if 'latitude' in shop and 'longitude' in shop and haversine(house_lat, house_lon, shop['latitude'], shop['longitude']) <= 1
    )
    Data_House.at[idx, 'd1000-7/11'] = count_7_eleven

    # นับจำนวนโรงเรียนในรัศมี 1 กิโลเมตร
    count_school = sum(
        1 for school_id, school in school_data.items()  # ใช้ .items() สำหรับ dict
        if 'latitude' in school and 'longitude' in school and haversine(house_lat, house_lon, school['latitude'], school['longitude']) <= 1
    )
    Data_House.at[idx, 'd1000-school'] = count_school

# แสดงผลข้อมูลหลังคำนวณ
Data_House[['id', 'd1000-7/11', 'd1000-school']]



Type of seven_data: <class 'dict'>
Type of school_data: <class 'dict'>


,id,d1000-7/11,d1000-school
0,1,0,0
1,4,0,0
2,16,0,0
3,18,0,0
4,19,0,0
...,...,...,...
1651,20460P,0,0
1652,20462P,0,0
1653,20463P,0,0
1654,20466P,0,0


In [39]:
import pandas as pd

# โหลดข้อมูลจากไฟล์ "คำตอบนักประเมิน(ID)"
df_evaluator = pd.read_excel("คำตอบนักประเมิน(ID).xlsx")

# กรองข้อมูลที่เกี่ยวข้องกับนักประเมิน ID=01
df_evaluator_01 = df_evaluator[df_evaluator['ID'] == 1]

# ตรวจสอบข้อมูล
print(df_evaluator_01.head())

# สมมุติว่า df_house คือ DataFrame ที่มีข้อมูลบ้าน
# ให้เพิ่มคอลัมน์ 'estimator' และ 'score' จากข้อมูลของนักประเมิน
Data_House = pd.read_excel("Data House.xlsx")  # โหลดข้อมูลบ้าน


# ตั้งชื่อคอลัมน์ที่ต้องการ
Data_House.rename(columns={'estimated_price': 'estimator', 'score': 'score'}, inplace=True)

# ตรวจสอบผลลัพธ์
Data_House


    ID ผู้ประเมิน                            post id ID property_type  \
74              1  https://nayoo.co/khonkaen/posts/1  1    บ้านเดี่ยว   
75              2  https://nayoo.co/khonkaen/posts/1  1    บ้านเดี่ยว   

   ความมั่นใจในการเมินราคาทรัพย์  ราคาทรัพย์  ประเมินราคาซื้อ-ขาย  \
74                        มั่นใจ    390000.0             600000.0   
75                        มั่นใจ    390000.0            1300000.0   

             ปัจจัยในการประเมินราคาซื้อ-ขาย เหตุผลไม่มั่นใจราคาประเมิน  \
74                 3,000-30,000 บาท/ตารางวา                        NaN   
75  ราคาเสนอขาย 1,000 - 3,000 บาท / ตารางวา                        NaN   

                      ให้คะแนนทรัพย์  
74  4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน  
75  4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน  


,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


####1.zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

In [30]:
import pandas as pd

# สมมุติว่า df_house คือ DataFrame ที่มีข้อมูลบ้าน
# df_house = pd.read_csv("Data House - บ้านมือ 2.csv")  # โหลดข้อมูลบ้าน

# 1. หาว่า zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด
average_sell_price_by_zone = Data_House.groupby('zone')['sell_price'].mean()
highest_avg_sell_price_zone = average_sell_price_by_zone.idxmax()
highest_avg_sell_price = average_sell_price_by_zone.max()

print(f"Zone ที่มีราคาซื้อขายบ้านเฉลี่ยสูงที่สุด: {highest_avg_sell_price_zone} ราคาเฉลี่ย: {highest_avg_sell_price:.2f}")

Zone ที่มีราคาซื้อขายบ้านเฉลี่ยสูงที่สุด: ในเมือง-รื่นรมย์-เทพารักษ์ ราคาเฉลี่ย: 7201666.67


In [43]:
print(Data_House.columns)


Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden'],
      dtype='object')


In [44]:
# สร้างคอลัมน์ใหม่ใน DataFrame
Data_House['d1000-7/11'] = 0
Data_House['d1000-school'] = 0

In [45]:
print(Data_House.columns)

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden',
       'd1000-7/11', 'd1000-school'],
      dtype='object')


####2.บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)

In [46]:
# 2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่
# จำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตร
Data_House['d1000-7/11'] = Data_House['d1000-7/11'].apply(lambda x: x if pd.notnull(x) else 0)

# หาบ้านที่มีจำนวน 7-11 ในรัศมี 1 กิโลเมตรมากที่สุด
max_7_11_count_house = Data_House[Data_House['d1000-7/11'] == Data_House['d1000-7/11'].max()]
max_7_11_count_house_price = max_7_11_count_house['sell_price'].iloc[0]

print(f"บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด มีราคาขาย: {max_7_11_count_house_price:.2f}")

บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด มีราคาขาย: 390000.00


In [48]:
print(Data_House.columns)

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden',
       'd1000-7/11', 'd1000-school'],
      dtype='object')


In [49]:
Data_House['score'] = 0

In [50]:
print(Data_House.columns)

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden',
       'd1000-7/11', 'd1000-school', 'score'],
      dtype='object')


####3.ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [56]:

# 3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด
average_score_by_sub_district = Data_House.groupby('sub_district')['score'].mean()
highest_avg_score_sub_district = average_score_by_sub_district.idxmax()
highest_avg_score = average_score_by_sub_district.max()

print(f"ตำบลที่ได้คะแนนทรัพย์เฉลี่ยจากนักประเมินมากที่สุด: {highest_avg_score_sub_district}")

ตำบลที่ได้คะแนนทรัพย์เฉลี่ยจากนักประเมินมากที่สุด: กุดกว้าง
